In [ ]:
import numpy as np # math library, simulation of random variables
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # more fancy stats visualization
from scipy import stats # stats library
from scipy import optimize # optimization library 
import statsmodels.api as sm # statistical modeling library

# Classical distributions

In python, we can use the library scipy as a probability toolbox, see https://docs.scipy.org/doc/scipy/reference/stats.html. The general use is to find the law that we want to use (a normal distribution in the example below) and to use the functions from the library. Each distribution is represented as an object, of which many attribute can be computed (see the documentation). For example law.cdf gives the CDF, law.pdf gives its density, and law.ppf is the inverse CDF (quantile function). For a discrete distribution, the density is replaced by the probability mass function (pmf). 

In [ ]:
from scipy.stats import norm
law = norm(2,3)
mean, var = law.stats(moments='mv')
print("Theorical mean: %.2F" %mean)
print("Theorical variance: %.2F" %var)

#let's plot the density
x = np.linspace(-6, 10, num=150)
plt.plot(x, law.pdf(x), label='norm pdf')

In [ ]:
# .. and the cdf 
plt.plot(x, law.cdf(x), label='norm cdf')

**Small exercice:** find the value $q_{\alpha}$ such that $\mathbb{P}_{Z \sim \mathcal{N}(0,1)}(Z \leq q_{\alpha}) = 1 - \alpha$ for $\alpha=0.025$.

Now let's take an example of discrete distribution that we saw in class: the Poisson distribution. The cells below visualizes the probability mass function ("density") of a Poisson distribution with parameter $\lambda = 3$ and its cdf.

In [ ]:
law = stats.poisson(3)
xplot = np.arange(20)
plt.stem(xplot,law.pmf(xplot),use_line_collection=True)

In [ ]:
xplot = np.linspace(0,20)
plt.plot(xplot,law.cdf(xplot))

In [ ]:
mean,variance = law.stats(moments='mv')
print("the mean is",mean,"and the variance is",variance)

**Small exercice:** for $Z \sim \mathcal{P}(3)$, find the values of $\mathbb{P}(Z=3), \mathbb{P}(Z \leq 2)$ and $\mathbb{P}(Z\geq 4)$

In [ ]:
print("P(Z = 3)=",## TO COMPLETE)
print("P(Z \leq 2)=",## TO COMPLETE)
print("P(Z \geq 4)=",## TO COMPLETE)

# Generating $n$-samples

The library numpy can simulate a sample from most usual probability laws. To generate a random sample, the algorithm depends on a sample from a uniform distribution $\mathcal{U}[0,1]$, this is the basic building block. To estimate a uniform distribution, one use a recurrent sequence and the difference between two terms of this sequence mimics the properties of i.i.d sample uniform law. As for all sequence, we need to initialize the sequence with a seed.

In [ ]:
n = 100 # Sample size
# initialize a random number generator
rng = np.random.RandomState(2024)
X = rng.normal(size=[n,2]) # random sampling from a standard normal distribution in dimension 2
plt.scatter(X[:,0],X[:,1]) # scatter plot of X

You can also directly call np.random.normal, but the seed will be changed each time you re-evaluate the cell. See the difference below. 

In [ ]:
X = np.random.normal(size=[n,2]) # random sampling 
plt.scatter(X[:,0],X[:,1]) # scatter plot of X

# Checking the adequation to a model

Below, we use np.random.exponential to sample $n=100$ points from an exponential distribution with mean $2$ (and therefore parameter $1/2$) and represent these points on the graph using the code below. You can execute several time the cell for several different samples to visualize the randomness.

In [ ]:
n=200
mean = 2
law = stats.expon(scale = mean)
mean,variance=law.stats(moments='mv')
print("mean is",mean,"and variance is",variance)

xplot = np.linspace(0.01,20,num=100)
plt.plot(xplot,law.pdf(xplot)) # plot the density

X = np.random.exponential(mean,size=n) # our n-sample
for Xi in X:
    plt.axvline(Xi, 0, 0.1, c='red', linewidth = 0.5) # we plot a vertical line for each sample point.

Using vertical bars on the x-axis to vizualize the data is not very nice, instead we can use an histogram that count how many sample are in a given bin and plot a bar with a height corresponding to the number of points in the bin (renormalized by the total number of sample). 

In [ ]:
plt.plot(xplot,law.pdf(xplot))
plt.hist(X, density=True,bins=15)

Now let's pretend that we observe the sample $X$ without knowing that it was generated from the distribution $\mathcal{E}(1/3)$. How do we check the adequation of this sample to this particular distribution? A first possibility is to look at the above figure and assess the closeness of the historgram and the density. Another possible way it to compare the empirical CDF $\widehat{F}_n$ to $F$, the cdf of the distribution that has generated the sample. Recall that the empirical cdf is defined as $$\widehat{F}_n(t) = \frac{1}{n}\sum_{i=1}^{n} \mathbb{1}(X_i \leq x)$$
and we know that if the sample $X$ comes from a distribution with cdf $F$, $\widehat{F}_n(t)$ converges almost surely to $F(t)$, for all $t\in \mathbb{N}$.

Let's verify this convergence graphically. The code below plots on the same graph the true CDF and the empirical CDF.  

In [ ]:
xplot = np.linspace(0,10,num=1000)

def empirical_cdf(t):
    return np.sum(X<=t)/n 

y_ecdf = [empirical_cdf(t) for t in xplot] # values of the empirical cdf on xplot
y_cdf = law.cdf(xplot) # values of the theoretical cdf on xplot

plt.plot(xplot,y_ecdf,label='Empirical cdf')
plt.plot(xplot,y_cdf,label='Theoretical cdf')
plt.legend()
plt.show()

Another way to assess the fit to a certain distribution is to use a so-called quantile-quantile plot (QQ-plot). The principle is to compare the empirical quantiles to the theoretical quantiles of the law. The empirical quantile of level $\alpha$, $\widehat{q}_{n,\alpha}$, for a sample $(X_1,\dots,X_n)$ is defined as the quantile of order $\alpha$ of the empirical cdf $\widehat{F}_n$. For all $\alpha$, the empirical quantile $\widehat{q}_{n,\alpha}$ is an estimator of the theoretical quantile, given by $F^{-1}(\alpha)$ if the sample is indeed drawn under the distribution $F$. 

The QQ-plot is the plot of the points $\left(\widehat{q}_{n,i},F^{-1}\left(\frac{i}{n+1}\right)\right)$ for $1\le i\le n$. We consider the distribution $F$ to be a good fit if these points are close to be aligned on the line $y=x$, as we should have $\widehat{q}_{n,i} \simeq F^{-1}\left(\frac{i}{n+1}\right)$ for all $i$. 

QQ-plot is implemented in the package statsmodel, see the [documentation](https://www.statsmodels.org/stable/generated/statsmodels.graphics.gofplots.qqplot.html) of the qqplot function. 

In [ ]:
sm.qqplot(X,law, line="45") 
plt.show()

# Comparing two estimators

Suppose that $X_1,\dots,X_n$ is an i.i.d sample drawn from some exponential distribution. 

We recall that the density of an exponential distribution with parameter $\theta$ is
$$f_{\theta}(x) = \theta \exp(-\theta x) \mathbb{1}_{[0,+\infty[}(x)$$
and its cdf is $$F_{\theta}(t) = (1 - \exp(-\theta t)) \mathbb{1}_{[0,+\infty[}(t).$$
We are interested in estimating the mean of this distribution, $\mu$, which satisfies $\mu = 1/\theta$.

A first obvious estimator is the moment estimator (which coincides with the MLE), $\widehat{\mu}_n = \frac{1}{n}\sum_{i=1}^{n} X_i$.

For the second estimator, we rely on the observation that the median of the exponential distribution can be easily related to the mean. Indeed, the median (of a continuous distribution) is the value $m$ such that $\mathbb{P}(X\leq m) = \frac{1}{2}$. Using the expression of the CDF, we have $m = \mu \log(2)$ and therefore we propose the estimator 
$$\widetilde{\mu}_n = \frac{\mathrm{median}(X_1,\dots,X_n)}{\log(2)} ,$$
where $\mathrm{median}(X_1,\dots,X_n)$ is the median of the empirical distribution. If $n$ is odd, its value is $X_{i_{med}}$ where the number of $X_i$'s strictly smaller than $X_{i_{med}}$ is equal to the number of $X_i$'s strictly greater than $X_{i_{med}}$.

We propose to compare these two estimators, first for a fixed sample size and then in an asymptotic regime. 

## Biais, variance and mean-square error

Computing the bias and variance of $\widehat{\mu}_n$ is quite straightfoward. As we saw multiple time in class for a $n$ sample, $\mathbb{E}[\widehat{\mu}_n] = \mathbb{E}[X_1]$ and $\mathrm{Var}[\widehat{\mu}_n] = \frac{\mathrm{Var}[X_1]}{n}$.

For $\widetilde{\mu}_n$, it is more sophisticated and we can prove that 
$$\mathbb{E}[\mathrm{median}(X_1,\dots,X_n)]=\mu\sum_{i=1}^{\lceil n/2\rceil}\frac{1}{n-i+1}\quad \text{and}\quad \mathrm{Var}(\mathrm{median}(X_1,\dots,X_n))=\mu^2\sum_{i=1}^{\lceil n/2\rfloor}\frac{1}{(n-i+1)^2}$$

**Compute the biais and variance of these two estimator, for $n = 51$ and $\mu=3$. Which one is better?** 

In [ ]:
mu = 3 
n = 51

bias1 = ## TO COMPLETE
variance1 = ## TO COMPLETE

bias2 = ## TO COMPLETE
variance2 = ## TO COMPLETE

print("moment estimator: biais",bias1,"and variance",variance1)
print("median-based estimator: biais",bias2,"and variance",variance2)

You may also compute the mean-square error

## Distribution of the estimators

It is possible go further than expectation-based metrics and to visualize (an approximation of) the *distributions* of the two estimators $\widehat{\mu}_n$ and $\widetilde{\mu}_n$, still for $n=51$ and when the underlying $n$-sample is generated from an exponential distribution with mean $\mu=3$.

To do so, we resort to simulation. We do $M$ simulations. In each simulation $i \leq M$, we begin by drawing a $n$-sample $X^{(i)}$ according to an exponential distribution of mean $\mu = 3$, then we compute the values of the estimators for this $X^{(i)}$, $\widehat{\mu}_n^{(i)}$ and $\widetilde{\mu}_n^{(i)}$. This provides $M$ samples $(\widehat{\mu}_n^{(1)},\dots,\widehat{\mu}_n^{(M)})$ from the distribution of $\widehat{\mu}_n$, and similarly $M$ samples from the distribution of $\widetilde{\mu}_n$.

We can then visualize the (empirical) distribution given by these samples, by looking at a histogram or a boxplot. 

In [ ]:
M = 100 # Number of different simulations
n = 51 # sample size for each simulation

mu = 3

samples_mu1 = []
samples_mu2 = []
for i in range(M):
    rng = np.random.RandomState(i) # choose a different seed for each experiment
    X = rng.exponential(mu, size=n) # generate a n-sample
    samples_mu1.append(np.mean(X)) # computing the first estimator and adding it to the list of samples
    samples_mu2.append(np.median(X)/np.log(2)) # computing the second estimator and adding it to the list of samples

In [ ]:
# box plot
plt.boxplot([samples_mu1,samples_mu2], labels=['$\widehat \mu_n$','$\widetilde \mu_n$'])
plt.show()

In [ ]:
# histogram + attempt at fitting a density (fancy seaborn visualization)
sns.distplot(samples_mu1)
sns.distplot(samples_mu2)
plt.axvline(mu, 0, 1, c='red', linewidth = 0.5)
plt.show()

## Asymptotic normality 

Clearly, the estimator $\widetilde{\mu}_n$ is worse than $\widehat{\mu}_n$, for a fixed sample size $n=51$. But could it still be somewhat good when the sample size goes large? Is it in particular asymptotically normal? This is what we try to answer here. 

**Using again $M$ simulations, generate $M$ samples from the distribution of $\sqrt{n}(\widetilde{\mu}_n- \mu)$ but for a larger value of $n$ and check whether the distribution looks Gaussian**.

In [ ]:
M = 1000
n = 500

samples = []
for i in range(M):
    pass 
    ## TO BE COMPLETED

sns.distplot(samples)

**How could you approximate its asymptotic variance? Compare it to that of $\widehat{\mu}_n$.**

Letting $Y = \sqrt{n}(\widetilde{\mu}_n - \mu)$, we are looking for $\sigma$ such that $Y/\sigma$ is a standard Gaussian. Letting $Z \sim \mathcal{N}(0,1)$ we can prove that the quantile functions satisfy $F_{Y}^{-1}(t) = \sigma F_Z^{-1}(t)$ for all $t$. Approximating $F_Y^{-1}(t)$ by the empirical quantiles, we can solve a regression problem.

# Maximum Likelihood and Optimization

The file "lifetimes.txt" contains some lifetime measures for $n=20$ industrial materials, the values are in thousands of hours. We import the dataset using the pandas library.

In [ ]:
df = pd.read_csv('lifetimes.txt')
df.head(5)

In [ ]:
df.describe()

In [ ]:
# the n-sample we are interested in
X=df['times']

In reliability studies, it is common to use a Weibull distribution (which is the generalization of an exponential distribution), whose density is given for some parameter $c>0$ by 
$$f_c(x)=cx^{c-1}\exp(-x^c)\mathbb{1}_{[0,+\infty[}(x)$$
In scipy.stats this distribution is called weibull_min

**Define a function called `weibull_neg_log_likelihood` that computes $-\log L(X;c)$ for a Weibull model from the $n$-sample $X$. Plot the negative log-likelihood (which is to be minimized) on $[0.1,5]$.**

In [ ]:
def weibull_neg_log_likelihood(c):
    return ## TO BE COMPLETED

xplot=np.linspace(0.1,5)
plt.plot(xplot,[weibull_neg_log_likelihood(x) for x in xplot])
plt.show()

This plot show that $-\log L(X;c)$ has a global minimum on $[0,5]$. To find it we could find the zero of the derivative analytically, but this is actually not that easy. Hence, we will instead use an iterative optimization algorithm implemented in the optimization package of scipy, called [minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). 

In [ ]:
from scipy.optimize import minimize 

initial_value = 5 # initial value required by the optimization algorithm
result = minimize(weibull_neg_log_likelihood,initial_value,options={ 'disp': True}) 
# do the minimization, display informations on the convergence.

In [ ]:
# result of minimize is an object with several attributes, including the minimizer itself
print(result)

In [ ]:
c_mle = result.x[0]
print("the mle is", c_mle)

**Do you think the learnt model fits well the data?**

You can visualize an histogram of your data together with the density of $f_{c_{mle}}$, compare the empirical cdf and the true cdf, or use a qqplot.

In [ ]:
plt.hist(X, density=True, bins=10)
xplot = np.linspace(np.min(X), np.max(X))
law = stats.weibull_min(c_mle) 
plt.plot(xplot,law.pdf(xplot))

In [ ]:
def empirical_cdf(t):
    return np.sum(X<=t)/20

xplot = np.linspace(0.1,2,num=100)
plt.plot(xplot,law.cdf(xplot),label="cdf of the optimal Weibull distribution")
plt.plot(xplot,[empirical_cdf(t) for t in xplot],label="empirical cdf")
plt.show()

In [ ]:
## qq-plot may be added 